# Imports 

In [ ]:
import os
import random 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Conv1D, Flatten, Dropout, Input, GlobalMaxPooling1D, concatenate

sacando codigos de folder

In [ ]:
path = "C:/Users/Alex/Documents/Cisco Config/Algorithms/Reto/reto/Dataset/version_2"

# Lista de carpetas en el dataset original
folder_group = [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]

# DataFrame 

In [ ]:
df = pd.DataFrame(columns=['Nombre1','Code1','Nombre2', 'Code2'])

In [ ]:
import os
import pandas as pd

path = "C:/Users/Alex/Documents/Cisco Config/Algorithms/Reto/reto/Dataset/version_2"

# Lista de carpetas en el dataset original
folder_group = [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]

df = pd.DataFrame(columns=['Nombre1','Code1','Nombre2', 'Code2'])

for folder in folder_group:
    dataset1 = []
    dataset2 = []

    folder_path = os.path.join(path, folder)
    problem_list = os.listdir(folder_path)

    # Eliminar extensión ".java" del nombre del archivo
    problem_list = [os.path.splitext(file)[0] for file in problem_list]

    # Leer el primer archivo .java con UTF-8
    with open(os.path.join(folder_path, problem_list[0] + '.java'), 'r', encoding='utf-8') as file1:
        dataset1.append(file1.read())
    d1 = ' '.join(dataset1)

    # Leer el segundo archivo .java con UTF-8
    with open(os.path.join(folder_path, problem_list[1] + '.java'), 'r', encoding='utf-8') as file2:
        dataset2.append(file2.read())
    d2 = ' '.join(dataset2)

    # Añadir datos al DataFrame
    df.loc[len(df.index)] = [problem_list[0], d1, problem_list[1], d2]


# Processamiento de Texto

In [ ]:
def load_data_from_directory(directory):
    texts = []
    labels = []
    # print(directory)
    for label in os.listdir(directory):
        # print(label)
        # class_dir = os.path.join(directory, label)
        class_dir = os.path.join(directory)
        # print(class_dir)
        if label.endswith(".txt"):
            file_path = os.path.join(class_dir, label)
            # print(file_path)
            with open(file_path, 'r', encoding='utf-8') as file:
                texts.append(file.read())
                labels.append(label)
                # print(texts)
    return texts, labels

# Ruta a la carpeta que contiene las subcarpetas 'java', 'python', y 'c'
data_directory = 'C:/Users/Alex/Documents/cisco Config/Algorithms/Reto/reto/reto'  # Ajusta esta ruta según sea necesario

# Cargar datos desde las carpetas de Java, Python y C
java_texts, java_labels = load_data_from_directory(os.path.join(data_directory, 'javaText'))
# python_texts, python_labels = load_data_from_directory(os.path.join(data_directory, 'pythonText'))
# c_texts, c_labels = load_data_from_directory(os.path.join(data_directory, 'cText'))

# Verificar que los datos se han cargado correctamente
print(f'Java texts: {len(java_texts)}')
# print(f'Python texts: {len(python_texts)}')
# print(f'C texts: {len(c_texts)}')

# Unir todos los textos y etiquetas
texts = java_texts #+ python_texts + c_texts
labels = java_labels #+ python_labels + c_labels

# Verificar los tamaños de los textos y etiquetas combinados
print(f'Total texts: {len(texts)}')
print(f'Total labels: {len(labels)}')


In [ ]:
import re

def remove_comments(text):
    # Remove all block comments (/* */)
    text = re.sub(r'/\*.*?\*/', '', text, flags=re.DOTALL)
    # Remove all line comments (//)
    text = re.sub(r'//.*', '', text)
    # Remove all Python line comments (#)
    text = re.sub(r'#.*', '', text)
    # Remove all Python block comments (''' ''')
    text = re.sub(r"'''(.*?)'''", '', text, flags=re.DOTALL)
    # Remove all Python block comments (""" """)
    text = re.sub(r'"""(.*?)"""', '', text, flags=re.DOTALL)
    
    # Split the text into lines, clean empty lines
    lines = text.split('\n')
    cleaned_lines = [line for line in lines if line.strip() != '']
    
    cleaned_text = '\n'.join(cleaned_lines)
    return cleaned_text

# Ejemplo de uso
code = """/* Block comment */
int x = 0; // Line comment

x++; /* Another block
comment */

System.out.println(x); // Print x

# This is a Python comment
y = 2 # Another Python comment

''' Python block comment
spanning multiple lines '''
z = 3; // C style line comment

"""
cleaned_code = remove_comments(code)
print(cleaned_code)


# Calculo de Similitudes

In [ ]:
def create_transition_matrix(text, words):
  # Inicializar la matriz de transición
  matrix = np.zeros((len(words), len(words)))

  # Dividir el texto en palabras
  word_list = text.split()

  # Calcular las frecuencias de las transiciones entre palabras
  for i in range(len(word_list) - 1):
      current_word = word_list[i]
      next_word = word_list[i + 1]

      current_index = words.index(current_word)
      next_index = words.index(next_word)

      matrix[current_index][next_index] += 1

  # Normalizar las filas de la matriz, evitando divisiones por cero
  row_sums = matrix.sum(axis=1)
  nonzero_rows = np.nonzero(row_sums)
  matrix_normalized = np.zeros_like(matrix)
  matrix_normalized[nonzero_rows, :] = matrix[nonzero_rows, :] / row_sums[nonzero_rows, np.newaxis]

  return matrix_normalized

In [ ]:
def get_file(file):
    return open(file, 'r', encoding='utf-8', errors='ignore').read()

In [ ]:
def split_words(text1, text2):
  splitted_words = list(set(text1.split() + text2.split()))
  return splitted_words

In [ ]:
def cos_ang(mat1, mat2):
  mat2T = np.transpose(mat2)
  c = np.dot(mat2T, mat1)
  prod_int = np.trace(c)
  normA = np.sqrt(np.trace((np.dot(np.transpose(mat1), mat1))))
  normB = np.sqrt(np.trace((np.dot(np.transpose(mat2), mat2))))

  return prod_int / (normA * normB)

In [ ]:
def get_words(text):
  file = open(text, "r", encoding = 'utf-8', errors ='ignore')
  words = file.read()
  file.close()

  punctuation = ["_", "-", ".", ":", "," , ";", "(", ")", "?", "¿", "¡", "!"]
  for i in punctuation:
      words = words.replace(i, " ")

  words = words.lower()
  words = words.replace('\n', ' ')

  return words

# Generar Tabla de Similitudes

In [ ]:
def similitud(text1, text2):
  data1 = remove_comments(text1)
  data2 = remove_comments(text2)

  split_data = split_words(data1, data2)

  mat1 = create_transition_matrix(data1, split_data)
  mat2 = create_transition_matrix(data2, split_data)
  cos = cos_ang(mat1, mat2)

  return cos # Debe retornar un valor %cos

In [ ]:
def get_two_random_texts(texts):

  if len(texts) <= 1:
    raise ValueError("The provided list of texts must contain at least two elements.")

  first_text_index = random.randint(0, len(texts) - 1)
  second_text_index = random.randint(0, len(texts) - 1)

  # Ensure distinct texts are returned
  while first_text_index == second_text_index:
    second_text_index = random.randint(0, len(texts) - 1)

  return texts[first_text_index],texts[second_text_index]


In [ ]:
similitud(get_two_random_texts(texts)[0], get_two_random_texts(texts)[1])

In [ ]:
df['Similitud'] = df.apply(lambda row: similitud(row['Code1'], row['Code2']), axis=1)
threshold = 0.8
df['Plagio'] = (df['Similitud'] > threshold).astype(int)
df

In [ ]:
df.to_csv('data.csv', index=False)

In [ ]:
# Visualizar la distribución de similitudes
plt.figure(figsize=(10, 6))
plt.hist(df['Similitud'], bins=20, color='blue', edgecolor='black')
plt.title('Distribución de Similitudes')
plt.xlabel('Similitud')
plt.ylabel('Frecuencia')
plt.show()

# Implementacion del Modelo

In [18]:


X = df[['Code1', 'Code2']]
y = df['Plagio'].to_numpy()

# Tokenización y padding
max_vocab_size = 10000
max_sequence_length = 1000

# Tokenizar los textos
tokenizer = Tokenizer(num_words=max_vocab_size, oov_token='<OOV>')

# Ajustar el tokenizador a los textos
tokenizer.fit_on_texts(pd.concat([df['Code1'], df['Code2']]))

# Convertir los textos a secuencias
X_code1_sequences = tokenizer.texts_to_sequences(df['Code1'])
X_code2_sequences = tokenizer.texts_to_sequences(df['Code2'])

# Padding de las secuencias
X_code1_padded = pad_sequences(X_code1_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
X_code2_padded = pad_sequences(X_code2_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# Combinar las secuencias padded
X_combined = np.concatenate([X_code1_padded, X_code2_padded], axis=1)



In [19]:
def quit_space(text):
    text = text.lower()
    text = text.replace('\n', ' ')
    text = text.replace('\t', ' ')
    return text

In [20]:
from sklearn.model_selection import train_test_split

df['Code1'] = df['Code1'].apply(quit_space)
df['Code2'] = df['Code2'].apply(quit_space)

x_features = df[['Code1', 'Code2']]
y_labels = df['Plagio']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_features, y_labels, test_size=0.2, random_state=42)

# Tokenización y padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['Code1'] + X_train['Code2'])

X_train_code1_sequences = tokenizer.texts_to_sequences(X_train['Code1'])
X_train_code2_sequences = tokenizer.texts_to_sequences(X_train['Code2'])
X_test_code1_sequences = tokenizer.texts_to_sequences(X_test['Code1'])
X_test_code2_sequences = tokenizer.texts_to_sequences(X_test['Code2'])

max_length = max(max(len(seq) for seq in X_train_code1_sequences), max(len(seq) for seq in X_train_code2_sequences))

X_train_code1_padded = pad_sequences(X_train_code1_sequences, maxlen=max_length, padding='post', truncating='post')
X_train_code2_padded = pad_sequences(X_train_code2_sequences, maxlen=max_length, padding='post', truncating='post')
X_test_code1_padded = pad_sequences(X_test_code1_sequences, maxlen=max_length, padding='post', truncating='post')
X_test_code2_padded = pad_sequences(X_test_code2_sequences, maxlen=max_length, padding='post', truncating='post')

print(max_length)


4711


In [21]:
from tensorflow.keras.models import Model

embedding_dim = 100
num_filters = 64
filter_sizes = [3, 4, 5]
dropout_rate = 0.5


input1 = Input(shape=(max_length,))
input2 = Input(shape=(max_length,))

embedding = Embedding(len(tokenizer.word_index) + 1, embedding_dim)

conv_blocks = []
for filter_size in filter_sizes:
    conv = Conv1D(filters=num_filters, kernel_size=filter_size, activation='relu')
    conv_block1 = conv(embedding(input1))
    conv_block2 = conv(embedding(input2))
    pool1 = GlobalMaxPooling1D()(conv_block1)
    pool2 = GlobalMaxPooling1D()(conv_block2)
    conv_blocks.extend([pool1, pool2])


if len(filter_sizes) > 1:
    merged = concatenate(conv_blocks)
else:
    merged = conv_blocks[0]

dropout = Dropout(0.5)(merged)
dense1 = Dense(64, activation='relu')(dropout)
output = Dense(1, activation='sigmoid')(dense1)

model = Model(inputs=[input1, input2], outputs=output)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
history = model.fit([X_train_code1_padded, X_train_code2_padded], y_train, epochs=10, batch_size=64, validation_data=([X_test_code1_padded, X_test_code2_padded], y_test))

Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 66s 5s/step - acc: 0.8143 - loss: 0.4896 - val_acc: 0.9563 - val_loss: 0.1846
Epoch 2/10
 6/12 ━━━━━━━━━━━━━━━━━━━━ 31s 5s/step - acc: 0.8957 - loss: 0.4158

KeyboardInterrupt: 

In [ ]:
# # Train the model
loss, accuracy = model.evaluate([X_test_code1_padded, X_test_code2_padded], y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)


In [ ]:
!pip install seaborn

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Predicciones en los datos de prueba
y_pred = model.predict([X_test_code1_padded, X_test_code2_padded])
y_pred_classes = np.round(y_pred)

# Crear la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred_classes)

print("Matriz de Confusión:")
print(conf_matrix)
